In [3]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

In [6]:
# loader
# use the data from documents to create vector store
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader('./data', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)

data = loader.load()

100%|██████████| 9/9 [00:00<00:00, 4500.33it/s]


In [7]:
# chunking the data to store in the database
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

documents = text_splitter.split_documents(data)

In [8]:
# embeddings from huggingface
from langchain.embeddings import HuggingFaceInstructEmbeddings
embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-large')

C:\Users\niran\anaconda3\envs\pytorch_learning\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
# embeddings from openai
# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [9]:
# loading vectors into VectorDB
from langchain.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore.save_local("vectorstore")

In [10]:
# load the vectorstore
vectorstore = FAISS.load_local("vectorstore", embeddings)

In [11]:
# Prompt to answer question
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant that answers the queries of the customer

{context}
Question: {question}
Answer here: """

PROMPT = PromptTemplate(
    template = prompt_template, input_variables=["context","question"]
)

In [12]:
# memory to store chat history
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

In [13]:
from langchain.llms import OpenAI, HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain

chain_type = {"prompt": PROMPT}
# to use open ai, uncomment it
# llm = OpenAI(openai_api_key=OPENAI_API_KEY)

# to use models from huggingface, uncomment this
llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs=chain_type
)

C:\Users\niran\anaconda3\envs\pytorch_learning\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [24]:
query="What are your pricing"


In [25]:
qa_chain({"question": query})

{'question': 'What are your pricing',
 'chat_history': [HumanMessage(content='How do I complain about the staff?'),
  AIMessage(content='If you are unhappy with your dining experience, please inform the waiter or hostess immediately. If you feel the waiter or hostess has not met your expectations, you can also contact the manager.'),
  HumanMessage(content='What are you open?'),
  AIMessage(content='We are open from 11:00 AM to 10:00 PM from Monday to Saturday and from 11:00 AM to 9:00 PM on Sundays.'),
  HumanMessage(content='What are you located?'),
  AIMessage(content='We are located at 123 Main Street, Downtown City. Our restaurant is easily accessible from major roads and public transport.'),
  HumanMessage(content='Summarize the questions I have asked till now'),
  AIMessage(content='If you are unhappy with your dining experience, please inform the waiter or hostess immediately. If you feel the waiter or hostess has not met your expectations, you can also contact the manager.'),
